# Advanced EDA: Amazon Product Reviews Sentiment

This notebook covers Phase 2 of the project: Advanced Exploratory Data Analysis. 

## Objectives
1. **Class Imbalance Analysis**: Visualize the distribution of star ratings.
2. **Metadata Mining**: Analyze sentiment trends by Brand and Category.
3. **Text Length Analysis**: Investigate the relationship between review length and rating.
4. **N-Gram Discovery**: Identify top keywords for Positive vs. Negative reviews.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Setup plotting aesthetics
sns.set_theme(style="whitegrid", palette="muted")
plt.rcParams["figure.figsize"] = (10, 6)

## 1. Load Data

In [ ]:
data_path = os.path.join('..', 'data', 'amazon_reviews.csv')
df = pd.read_csv(data_path, low_memory=False)

print(f"Dataset Shape: {df.shape}")
df.head()